<a href="https://colab.research.google.com/github/504aldo/gis/blob/main/GIS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Mapas Interactivos**
---


---

Esta sección es texto, solo formateo como código.



```
# print(2+2)
```






Esta sección es código SIN ejecución


In [ ]:
print(3*5)

Esta sección es código que se ejecuta

In [ ]:
print(2+2)

4


# 2. Mapa Interactivo

Este es un bloque de texto.
El siguiente es un bloque de código donde muestro código y resultados para el mapa interactivo

In [39]:
# Montar Google Drive en el entorno de Google Colab
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

# Trabajar con geometrias (shapefiles) y mapas
import geopandas as gpd
import matplotlib.pyplot as plt

# Interactividad
import folium
import webbrowser # abrir mapa guardado en html
import os

# Corregir puntos de lat y lon atipicos
from shapely.geometry import box

# Agregar la ruta a sys.path para importar funciones
import sys
sys.path.append('/content/drive/MyDrive/Colab Notebooks')

  # Funciones específicas de geo_funciones.py
  from geo_funciones import crear_mapa_agregado    # mapa agregado con puntos


Mounted at /content/drive


leer

In [42]:
# Cargar archivos shapefile (País)
pais_shp = "/content/drive/MyDrive/Colab Notebooks/shp_HN/hnd_admbnda_adm0_sinit_20161005.shp"
pais_gdf = gpd.read_file(pais_shp)

# Cargar archivos shapefile (Departamentos)
departamentos_shp = "/content/drive/MyDrive/Colab Notebooks/shp_HN/hnd_admbnda_adm1_sinit_20161005.shp"
departamentos_gdf = gpd.read_file(departamentos_shp)

# Cargar archivos shapefile (Municipios)
municipios_shp = "/content/drive/MyDrive/Colab Notebooks/shp_HN/hnd_admbnda_adm2_sinit_20161005.shp"
municipios_gdf = gpd.read_file(municipios_shp)

# Cargar archivos shapefile (Aldeas=3730)
aldeas_shp = "/content/drive/MyDrive/Colab Notebooks/shp_HN/hnd_admbnda_adm3_sinit_20161005.shp"
aldeas_gdf = gpd.read_file(aldeas_shp)

# Cargar archivos shapefile (Caseríos=27969, censo2001=30810, censo2013=30012)
caserios_shp = "/content/drive/MyDrive/Colab Notebooks/shp_HN/hnd_pplp_sinit_20160125.shp"
caserios_gdf = gpd.read_file(caserios_shp)



limpieza

In [43]:
# Columnas a borrar
to_del = ["shape_leng", "shape_area", "adm0_ref", "adm1_ref", "adm2_ref", "date", "validon", "validto"]

# Diccionario de nombres de columnas y nuevos nombres
cols_rename = {
    "adm0_pcode": "admin0pcod",
    "adm0_es": "admin0name",
    "adm1_pcode": "admin1pcod",
    "adm1_es": "admin1name",
    "adm2_pcode": "admin2pcod",
    "adm2_es": "admin2name",
    "pcode": "pcod",
    }

# Limpiar columnas
for df in [pais_gdf, departamentos_gdf, municipios_gdf, aldeas_gdf, caserios_gdf]:
    df.columns = df.columns.str.lower()  # Convierte nombres a minúsculas
    cols_to_drop = [col for col in to_del if col in df.columns]  # Encuentra las columnas a eliminar que existen
    cols_to_drop.extend(df.select_dtypes(include=['float64']).columns.tolist())  # agregar a cols_to_drop columnas de tipo float64
    df.drop(columns=cols_to_drop, inplace=True)  # Elimina las columnas encontradas en 'cols_to_drop'
    df.rename(columns=cols_rename, inplace=True) # Renombra las columnas de acuerdo a diccionario

# Crear una máscara booleana, con bounding box de Honduras, y aplicar
mask = caserios_gdf.geometry.within(box(*pais_gdf.total_bounds)) # * para desempaquetar los valores
caserios_gdf.loc[~mask, ['geometry']] = None                     # Asignar NaN


verificar

In [44]:
# Corregido caserios_gdf (valores de lat y lon)
print(caserios_gdf[caserios_gdf["admin2name"] == "Guanaja"].tail())


       objectid_1       featurenam         pcod       featureref  popplacecl  \
15809       15810   Qda. El LimÃ³n  HN110204018    Qda. El Limon           4   
15810       15811     Stuart's Cay  HN110204020      Stuarts Cay           4   
15811       15812  Brink Point # 2  HN110204023  Brink Point # 2           4   
15812       15813      Rosina Bill  HN110204024      Rosina Bill           4   
15813       15814        Joshs Cay  HN110204025        Joshs Cay           4   

      popplace_1 admin2name admin2pcod         admin1name admin1pcod  \
15809   CaserÃ­o    Guanaja     HN1102  Islas de La Bahia       HN11   
15810   CaserÃ­o    Guanaja     HN1102  Islas de La Bahia       HN11   
15811   CaserÃ­o    Guanaja     HN1102  Islas de La Bahia       HN11   
15812   CaserÃ­o    Guanaja     HN1102  Islas de La Bahia       HN11   
15813   CaserÃ­o    Guanaja     HN1102  Islas de La Bahia       HN11   

      admin0name admin0pcod                    geometry  
15809   Honduras         HN 

mapa

In [ ]:
# Crear un mapa centrado en Atlántida
mapa = folium.Map(location=[15.7835, -86.7825], zoom_start=10)

# Filtrar los datos de los municipios y caseríos de Atlántida
municipios_atlantida = municipios_gdf[municipios_gdf['admin1pcod'] == 'HN01']
caserios_atlantida = caserios_gdf[caserios_gdf['admin1pcod'] == 'HN01']

# Añadir los municipios al mapa
for _, municipio in municipios_atlantida.iterrows():
    folium.GeoJson(municipio['geometry']).add_to(mapa)

# Añadir los caseríos al mapa
for _, caserio in caserios_atlantida.iterrows():
    if not caserio['geometry'] is None:
        folium.Marker(location=[caserio['geometry'].y, caserio['geometry'].x], popup=caserio['featurenam']).add_to(mapa)

# Mostrar el mapa
mapa
